# Tracks preprocessing

В данном ноутбуке выполняется предобработка музыкальных отрывков для их будущей классификации по жанрам (блюз, поп, диско, рок и т.д.). Из каждого музыкального отрывка выделяется основной признак, на основе которого будет выполнена классификация, - мел-частотные кепстральные коэффициенты.
Появилось предположение, что такой метод будет более точным нежели метод на основе свёрточных нейронных сетей, примененных к спектрограммам, поскольку он имеет ФИЗИЧЕСКУЮ обоснованность.
Проверим.

In [1]:
import os
import json
import sklearn
import numpy as np
import librosa as lr
import librosa.display
import IPython.display as ipd
import matplotlib.pyplot as plt

In [2]:
# загрузка музыкального трека из указанного пути

def get_data(path):
    signal, sr = lr.load(path)
    return signal, sr

In [3]:
# заполнение словаря соответствия жанров и их идентификаторов

def fill_genres(path):
    genre_list = os.listdir(path)
    return dict(zip(genre_list, np.arange(0, len(genre_list)).tolist()))

In [4]:
# получение жанра трека или идентификатора

def get_genre_or_label(path, vocab, flag):
    if flag == 'label':
        return vocab.get(path.split('\\')[5])
    else:
        return path.split('\\')[5]

In [5]:
# получение мел-частотных кепстральных коэффициентов для трека

def get_mfcc(file, n_fft, hop_length, n_mfcc):
    mfcc = lr.feature.mfcc(get_data(file)[0], n_fft=n_fft, hop_length=hop_length, n_mfcc=n_mfcc)
    mfcc = mfcc.T
    
    return mfcc.tolist()

In [6]:
# добавление MFCC и кодов жанров в словарь

def get_files(path, number, vocab, genres_vocab):
    for (address, dirs, files) in os.walk(path):
        for ind, file in enumerate(files):
            if ind < number:
                vocab['mfcc'].append(get_mfcc(address + '\\' + file, 2048, 512, 13))
                vocab['labels'].append(get_genre_or_label(address + '\\' + file, genres_vocab, flag='label'))

In [7]:
PATH = "E:\\Data Science\\Datasets\\Music Genre Classification\\genres_original\\"

In [8]:
data = {'mfcc' : [], 
       'labels' : []}

In [9]:
genres = fill_genres(PATH)

genres

{'blues': 0,
 'classical': 1,
 'country': 2,
 'data.json': 3,
 'disco': 4,
 'hiphop': 5,
 'jazz': 6,
 'metal': 7,
 'pop': 8,
 'reggae': 9,
 'rock': 10}

In [11]:
get_files(PATH, 10, data, genres)

In [24]:
# заполненный словарь запишем в json файл для хранения

with open(PATH + 'data.json', "w") as f_data:
    json.dump(data, f_data, indent=4)